In [2]:
# 0314 v2 - 가까운 두 개 마커로 위치 파악
# 마커 개수 늘림 시야에 3개까지 잘 잡히는데 가까운 2개를 고르려다 보니까 어떤 마커 2개를 고르는지에 따라서 좌표가 다르게 나타남
#v3: 여러개 중에 2개씩 이용해서 위치 뽑고 평균 내기
#v4: 칼만 필터  - v3보다 개선됨 (튀는 값이 줄어들었지만 여전히 많긴 함)

# 0317 v1: z축 거리 고정해서 입력하기 + 칼만필터까지 적용
# 정확도 높음 z값 -1.8m로 했는데 로봇에 부착하고 다시 거리 재보면 될듯
# v2: 마커 17번 빠졌던 거 추가

In [2]:
import cv2
import cv2.aruco as aruco
import numpy as np

# ===========================
# 📌 1. 설정 및 초기값
# ===========================
cameraMatrix = np.load('C:/cal_cam/calibration_matrix.npy')
distCoeffs = np.load('C:/cal_cam/distortion_coefficients.npy')
marker_length = 0.16  # 마커 실제 길이 (m)
FIXED_Z =-1.8  # 고정된 Z 값 (m), 예: 천장 높이

# 마커 월드 좌표 (예시)
marker_world_pos = {
    0: np.array([0.0, 0.0, 0.0]), 1: np.array([1.5, 0.0, 0.0]),
    2: np.array([3.0, 0.0, 0.0]), 3: np.array([4.5, 0.0, 0.0]),
    4: np.array([0.0, 1.5, 0.0]), 5: np.array([1.5, 1.5, 0.0]),
    6: np.array([3.0, 1.5, 0.0]), 7: np.array([4.5, 1.5, 0.0]),
    8: np.array([0.0, 3.0, 0.0]), 9: np.array([1.5, 3.0, 0.0]),
    10: np.array([3.0, 3.0, 0.0]), 11: np.array([4.5, 3.0, 0.0]),
    12: np.array([0.7, 0.7, 0.0]), 13: np.array([2.3, 0.7, 0.0]),
    14: np.array([3.7, 0.7, 0.0]), 15: np.array([0.7, 2.3, 0.0]),
    16: np.array([2.3, 2.3, 0.0]), 17: np.array([3.7, 2.3, 0.0])
}

# 칼만 필터 초기화 함수
def init_kalman_filter():
    kf = cv2.KalmanFilter(6, 3)  # (상태 6: 위치+속도, 관측 3: 위치)
    kf.measurementMatrix = np.eye(3, 6, dtype=np.float32)
    kf.transitionMatrix = np.array([
        [1, 0, 0, 1, 0, 0],
        [0, 1, 0, 0, 1, 0],
        [0, 0, 1, 0, 0, 1],
        [0, 0, 0, 1, 0, 0],
        [0, 0, 0, 0, 1, 0],
        [0, 0, 0, 0, 0, 1],
    ], dtype=np.float32)
    kf.processNoiseCov = np.eye(6, dtype=np.float32) * 1e-4
    kf.measurementNoiseCov = np.eye(3, dtype=np.float32) * 1e-2
    return kf

kalman_filters = {}  # 마커별 칼만 필터 저장

# ===========================
# 📌 2. Z 고정 기반 카메라 위치 추정 함수
# ===========================
def compute_camera_pos_fixed_z(marker_id, rvec, tvec, marker_world, fixed_z=FIXED_Z):
    R, _ = cv2.Rodrigues(rvec)
    t = tvec.reshape(3, 1)
    t[2] = fixed_z  # z 고정
    cam_in_marker = -R.T @ t  # 마커 기준에서 본 카메라 위치
    return marker_world.reshape(3, 1) + cam_in_marker  # 월드 좌표 기준

# ===========================
# 📌 3. 카메라 열기 및 메인 루프
# ===========================
cap = cv2.VideoCapture(1)
cap.set(cv2.CAP_PROP_FRAME_WIDTH, 1280)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 720)
cap.set(cv2.CAP_PROP_EXPOSURE, -6)

aruco_dict = aruco.getPredefinedDictionary(aruco.DICT_4X4_50)
detector_params = aruco.DetectorParameters()

if not cap.isOpened():
    print("카메라 열기 실패!")
    exit()

while True:
    ret, frame = cap.read()
    if not ret:
        print("프레임 읽기 실패!")
        break

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    corners, ids, _ = aruco.detectMarkers(gray, aruco_dict, parameters=detector_params)

    if ids is not None:
        aruco.drawDetectedMarkers(frame, corners, ids)
        rvecs, tvecs, _ = aruco.estimatePoseSingleMarkers(corners, marker_length, cameraMatrix, distCoeffs)

        id_to_pose = {id_[0]: (rvecs[i], tvecs[i]) for i, id_ in enumerate(ids)}
        found_ids = ids.flatten()

        cam_positions = []
        weights = []

        for m_id in found_ids:
            if m_id in marker_world_pos:
                rvec, tvec = id_to_pose[m_id]
                world_pos = marker_world_pos[m_id]

                # 고정된 Z로 카메라 위치 계산
                cam_pos = compute_camera_pos_fixed_z(m_id, rvec, tvec, world_pos).flatten()

                # 칼만 필터 초기화 및 보정
                if m_id not in kalman_filters:
                    kalman_filters[m_id] = init_kalman_filter()
                kf = kalman_filters[m_id]
                kf.correct(cam_pos.astype(np.float32))  # 관측값 보정
                estimate = kf.predict()[:3]  # 예측 위치

                cam_positions.append(estimate)
                distance = np.linalg.norm(tvec)  # 마커와의 거리
                weights.append(1.0 / (distance + 1e-6))  # 가까운 마커에 가중치

                # 화면에 ID 표시
                c = corners[found_ids.tolist().index(m_id)][0][0]
                cv2.putText(frame, f"ID {m_id}", (int(c[0]), int(c[1] - 10)),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)

        # 가중 평균으로 카메라 위치 결정
        if cam_positions:
            weights = np.array(weights)
            weights /= np.sum(weights)  # 정규화
            cam_positions = np.array(cam_positions)
            camera_world_pos = np.average(cam_positions, axis=0, weights=weights)

            # 카메라 위치 출력
            cx, cy, cz = camera_world_pos.ravel()
            print(f"[INFO] Camera world position (filtered): ({cx:.2f}, {cy:.2f}, {cz:.2f})")

    else:
        print("❌ 마커 감지 실패")

    # 화면 표시
    cv2.imshow("Z-Fixed ArUco with Kalman", frame)
    if cv2.waitKey(1) & 0xFF == 27:  # ESC 키 종료
        break

cap.release()
cv2.destroyAllWindows()


[INFO] Camera world position (filtered): (0.00, 0.00, 0.00)
[INFO] Camera world position (filtered): (0.02, 0.01, -0.02)
[INFO] Camera world position (filtered): (0.12, 0.05, -0.09)
[INFO] Camera world position (filtered): (0.40, 0.15, -0.28)
[INFO] Camera world position (filtered): (0.92, 0.35, -0.64)
[INFO] Camera world position (filtered): (1.61, 0.59, -1.12)
[INFO] Camera world position (filtered): (2.24, 0.87, -1.56)
[INFO] Camera world position (filtered): (3.17, 1.62, -1.89)
[INFO] Camera world position (filtered): (3.07, 1.24, -2.08)
[INFO] Camera world position (filtered): (3.30, 1.27, -2.25)
[INFO] Camera world position (filtered): (3.39, 1.30, -2.32)
[INFO] Camera world position (filtered): (3.42, 1.32, -2.32)
[INFO] Camera world position (filtered): (3.41, 1.37, -2.26)
[INFO] Camera world position (filtered): (3.30, 1.32, -2.21)
[INFO] Camera world position (filtered): (3.15, 1.26, -2.14)
[INFO] Camera world position (filtered): (3.01, 1.21, -2.07)
[INFO] Camera world posit